<a href="https://colab.research.google.com/github/Aman-Agrawal01/Quantum-Stuffs/blob/main/Missing_Number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Task 2 Missing Number**
From a function that has as a parameter a vector of positive integers of size 2^n -1, which is missing a number, this vector can be disordered, to search for the missing number from a quantum circuit.



```
def missing_number(Optional[list,array, str]: input_vector):
     “””
input_vector: List, array or string that contain integer values of size 2^n -1, where are missing a number to obtain all the number 2^n 
Return the positive integer value that is missing in the input.
     “””

     # use a framework that works with quantum circuits, qiskit, cirq, pennylane, etc. 
      # define a quantum circuit to convert the vector in a quantum circuit
     # define an oracle to find the missing value
    # encoding the output value in an integer value

# consider print your quantum circuit,

```

Example:
```
A =  missing_number([2,0,1])
print(A)
3
```



Importing Necessary Libraries 


In [ ]:
import time
import qiskit
import numpy as np
from qiskit.circuit.library import Diagonal
from qiskit.quantum_info import Statevector

A classical Algorithm that finds the missing number 

In [ ]:
def missing_number_classical(input_vector):
  size = len(input_vector)
  for i in range(size+1):
    if i in input_vector :
      continue
    else :
      return i

In [ ]:
input_vector = np.random.choice(range(16), 15, replace=False)

print("Vector is ",input_vector)
print("Missing Number is",missing_number_classical(input_vector))

Vector is  [12 14  5 11  7  9  2  0 15 10  8  6 13  1  4]
Missing Number is 3


To find the missing number, I have used a quantum subroutine known as amplitude amplification. Created an oracle that gives us access to the function. The function gives true value if input is the missing number else false. Mathematically, it is as show below - <br> <br>
$\begin{equation}
f(x) :=
    \begin{cases}
        1 & \text{if } x \text{ is the missing element}\\
        0 & \text{otherwise}
    \end{cases}
\end{equation}$
<br> <br>
The Oracle $O_f$ to this function gives a negative phase to the missing element state while other states remain as it is.  <br> <br>
$
O_f := \begin{bmatrix}
(-1)^{f(0)} & 0 & 0 & . & 0 \\
0 & (-1)^{f(1)} & 0 & . & 0 \\
. & 0 &  & . & 0 \\
. & . & . & . & . \\
0 & 0 & 0 & . &  (-1)^{f(2^n-1)}\\
\end{bmatrix}
$ <br> <br>
where $2^n - 1$ is the size of the vector. After this, we add an amplitude amplification gates which amplifies the amplitude of the missing number. So, in each step or iteration we add oracle followed by the amplification gates. The algorithm takes nearly $\lfloor \frac{\pi}{4}2^{n/2} ⌋$ steps or iterations.

In [ ]:
def missing_number_quantum(input_vector):

  if(len(input_vector))

  """Transforming the integer array to corresponding binary valued arrays with fixed lengths
    Example - [2,1,0] -> ['10','01','00'] """
  num_qubits = int(np.log2(len(input_vector)+1))
  binary_input_vector = list()
  for i in range(len(input_vector)):
    binary_input_vector.append(bin(input_vector[i]).replace('0b','').zfill(num_qubits))

  """ Creating Oracle which results in adding a negative phase to the solution states """
  mark_state = Statevector.from_label(bin(missing_number_classical(input_vector)).replace('0b','').zfill(num_qubits))
  oracle = Diagonal((-1)**mark_state.data) 

  " Creating the Quantum Circuit "
  qc = qiskit.QuantumCircuit(num_qubits+1,num_qubits)

  " Equal superposition in the first n qubits and |-> state in auxiliary/ancilia bit"
  qc.x([num_qubits])
  qc.h([i for i in range(num_qubits+1)])
  qc.barrier()

  " Grover search requires pi*sqrt(n)/4 steps/iterations "
  steps = int(np.pi*np.sqrt(2**num_qubits)/4)

  for n in range(steps):

    qc.append(oracle,range(num_qubits))

    qc.barrier()

    "Adding the Amplification"
    qc.h(range(num_qubits))
    qc.x(range(num_qubits))
    qc.mcx([i for i in range(num_qubits)],num_qubits)
    qc.x(range(num_qubits))  
    qc.h(range(num_qubits))

    qc.barrier()

  qc.measure(range(num_qubits),range(num_qubits))
  qc.draw(output='text')

  aer_sim = qiskit.Aer.get_backend('aer_simulator')
  circuit = qiskit.transpile(qc, aer_sim)
  qobj = qiskit.assemble(circuit)
  results = aer_sim.run(qobj).result()
  counts = results.get_counts()

  return int(max(counts, key=counts.get),2),qc.draw(output='text')

In [ ]:
print("Missing number according to Amplitude Amplification is",missing_number_quantum(input_vector)[0])
print("The corresponding quantum circuit for the problem is ", missing_number_quantum(input_vector)[1])
print("Does both the method gives same results ? - ", missing_number_quantum(input_vector)[0] == missing_number_classical(input_vector))

Missing number according to Amplitude Amplification is 3
The corresponding quantum circuit for the problem is       ┌───┐      ░ ┌───────────┐ ░ ┌───┐┌───┐     ┌───┐┌───┐ ░ ┌───────────┐ ░ »
q_0: ┤ H ├──────░─┤0          ├─░─┤ H ├┤ X ├──■──┤ X ├┤ H ├─░─┤0          ├─░─»
     ├───┤      ░ │           │ ░ ├───┤├───┤  │  ├───┤├───┤ ░ │           │ ░ »
q_1: ┤ H ├──────░─┤1          ├─░─┤ H ├┤ X ├──■──┤ X ├┤ H ├─░─┤1          ├─░─»
     ├───┤      ░ │  Diagonal │ ░ ├───┤├───┤  │  ├───┤├───┤ ░ │  Diagonal │ ░ »
q_2: ┤ H ├──────░─┤2          ├─░─┤ H ├┤ X ├──■──┤ X ├┤ H ├─░─┤2          ├─░─»
     ├───┤      ░ │           │ ░ ├───┤├───┤  │  ├───┤├───┤ ░ │           │ ░ »
q_3: ┤ H ├──────░─┤3          ├─░─┤ H ├┤ X ├──■──┤ X ├┤ H ├─░─┤3          ├─░─»
     ├───┤┌───┐ ░ └───────────┘ ░ └───┘└───┘┌─┴─┐└───┘└───┘ ░ └───────────┘ ░ »
q_4: ┤ X ├┤ H ├─░───────────────░───────────┤ X ├───────────░───────────────░─»
     └───┘└───┘ ░               ░           └───┘           ░               ░ »
c: 4/════

## BONUS 
Which is the largest list that can be implemented? Identify it and describe the result


I believe the problem is bit ambiguous or incomplete. It should be like which is the largest list that can be implemented **efficiently**. I iterate the size of input vector and noted their time using time library in Python as shown below in the code. It seems like largest list that can be implemented **efficiently** is of size $2^{10} - 1$ which takes around a minute. After that, the system stops responding.


In [ ]:
for m in range(2,13):
  n = 2**m
  input_vector = np.random.choice(range(n), n-1, replace=False)
  start_time = time.time()
  missing_number_quantum(input_vector)
  print("m =", m, "takes %s seconds" % (time.time() - start_time))

m = 2 takes 0.05356597900390625 seconds
m = 3 takes 0.16853117942810059 seconds
m = 4 takes 0.3633854389190674 seconds
m = 5 takes 0.7449326515197754 seconds
m = 6 takes 1.7278356552124023 seconds
m = 7 takes 5.205981969833374 seconds
m = 8 takes 13.989250659942627 seconds
m = 9 takes 28.49116063117981 seconds
m = 10 takes 86.62120294570923 seconds
m = 11 takes 246.16140413284302 seconds
m = 12 takes 680.7665221691132 seconds
